# Homework 4 Final Models (Text Classification)

Adam Kiehl  
5/7/2023

In [20]:
# import analysis packages
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Embedding, Flatten, SimpleRNN, TextVectorization
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score

### Data Cleaning

In [21]:
# read data from .csv files
trainDF = pd.read_csv('./ibotta_train.csv')
testDF = pd.read_csv('./ibotta_test.csv')

# combine data sets for preprocessing
trainDF['origin'] = 'train'
testDF['origin'] = 'test'
fullDF = pd.concat([trainDF, testDF])

# text cleaning
fullDF['Brand_name'].where(-fullDF['Brand_name'].isna(), '', inplace = True)
fullDF['Brand_name'] = fullDF['Brand_name'].apply(lambda x: x.lower().replace("'", "").replace(",", "").replace(":", "").replace("-", "").replace(".", ""))
fullDF['Name'] = fullDF['Name'].apply(lambda x: x.lower().replace("'", "").replace(",", "").replace(":", "").replace("-", "").replace(".", ""))

# combine brand and name fields
fullDF['brandAlready'] = fullDF.apply(lambda x: x['Name'].find(x['Brand_name']), axis = 1)
fullDF.loc[fullDF.brandAlready == -1, 'Name'] = fullDF.loc[fullDF.brandAlready == -1, 'Brand_name'] + \
    ' ' + fullDF.loc[fullDF.brandAlready == -1, 'Name']
fullDF.drop('brandAlready', axis = 1, inplace = True)

# split data
trainDF = pd.DataFrame(fullDF.loc[fullDF['origin'] == 'train'].drop('origin', axis = 1))
testDF = pd.DataFrame(fullDF.loc[fullDF['origin'] == 'test'].drop(['origin', 'Category'], axis = 1))

### Text Vectorization

In [22]:
# find total number of unique words
unique_words = np.unique(np.array(' '.join(np.array(fullDF['Name'])).split(' ')))
max_length = len(unique_words)

# initialize empty dataframe
wordBag = pd.DataFrame(np.zeros((len(fullDF), max_length)), 
                       columns = unique_words)

# loop through product names
for i, productName in enumerate(fullDF['Name'].apply(lambda x: x.split(' '))):
    # loop through words in name
    for word in productName:
        # identify word presence
        wordBag.loc[i, word] = 1

# split word bag
trainDFwordBag = wordBag.loc[0:7999]
testDFwordBag = wordBag.loc[8000:9999]

In [23]:
# keras tokenizer function
def textVecGen(size):
    # train keras tokenizer
    tokenizer = TextVectorization(max_tokens = size, 
                                  output_sequence_length = size)
    tokenizer.adapt(fullDF['Name'])

    # vectorize data
    textVecDF = pd.DataFrame(tokenizer(fullDF['Name']))
    
    # split vectorized data
    trainDFtextVec = textVecDF.loc[0:7999]
    testDFtextVec = textVecDF.loc[8000:9999]

    return(trainDFtextVec, testDFtextVec)

# generate vectorized data
trainDFtextVec1000, testDFtextVec1000 = textVecGen(1000)

### Model Fitting

In [24]:
# set random seeds
np.random.seed(542023)
tf.random.set_seed(542023)

# define model architecture
model1 = Sequential([
    Dense(512, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(7, activation = 'softmax')
])

# define F1 metric
f1_score_metric = F1Score(num_classes = 7, average = 'weighted')

# compile model
model1.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy', f1_score_metric])
    
# define early stopping criterion
early = EarlyStopping(monitor = 'f1_score', mode = 'max', patience = 3)

# train deep learning model
trained1 = model1.fit(trainDFwordBag,
                      to_categorical(trainDF['Cat_code']),
                      epochs = 100,
                      batch_size = 128,
                      callbacks = early,
                      verbose = 1)

# predict on test set
pred1 = model1.predict(testDFwordBag)

# create submission data frame
submission = pd.DataFrame({'Id': testDF['Id'], 'Cat_code': np.argmax(pred1, axis = 1).reshape(len(pred1), )})

# export submission
submission.to_csv('./submission1.csv', index = False)

Epoch 1/100
63/63 [==============================] - 2s 23ms/step - loss: 0.6617 - accuracy: 0.7756 - f1_score: 0.7751
Epoch 2/100
63/63 [==============================] - 1s 19ms/step - loss: 0.1072 - accuracy: 0.9715 - f1_score: 0.9715
Epoch 3/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0493 - accuracy: 0.9856 - f1_score: 0.9856
Epoch 4/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0291 - accuracy: 0.9904 - f1_score: 0.9904
Epoch 5/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0172 - accuracy: 0.9946 - f1_score: 0.9946
Epoch 6/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0131 - accuracy: 0.9946 - f1_score: 0.9946
Epoch 7/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0089 - accuracy: 0.9964 - f1_score: 0.9964
Epoch 8/100
63/63 [==============================] - 1s 19ms/step - loss: 0.0076 - accuracy: 0.9966 - f1_score: 0.9966
Epoch 9/100
63/63 [=============================

In [26]:
# set random seeds
np.random.seed(542023)
tf.random.set_seed(542023)

# define model architecture
model2 = Sequential([
    Embedding(1000, 128, input_shape = (1000, )),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(7, activation = 'softmax')
])

# print model summary
model2.summary()

# define F1 metric
f1_score_metric = F1Score(num_classes = 7, average = 'weighted')

# compile model
model2.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy', f1_score_metric])
    
# define early stopping criterion
early = EarlyStopping(monitor = 'f1_score', mode = 'max', patience = 3)

# train deep learning model
trained2 = model2.fit(trainDFtextVec1000,
                      to_categorical(trainDF['Cat_code']),
                      epochs = 100,
                      batch_size = 32,
                      callbacks = early,
                      verbose = 1)

# predict on test set
pred2 = model2.predict(testDFtextVec1000)

# create submission data frame
submission = pd.DataFrame({'Id': testDF['Id'], 'Cat_code': np.argmax(pred2, axis = 1).reshape(len(pred2), )})

# export submission
submission.to_csv('./submission2.csv', index = False)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 128)         128000    
                                                                 
 flatten_1 (Flatten)         (None, 128000)            0         
                                                                 
 dense_28 (Dense)            (None, 128)               16384128  
                                                                 
 dense_29 (Dense)            (None, 64)                8256      
                                                                 
 dense_30 (Dense)            (None, 7)                 455       
                                                                 
Total params: 16,520,839
Trainable params: 16,520,839
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
250/250 [=====================

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x3ee686220>
    label = <none> 
    device = <AGXG13GDevice: 0x137dc5a00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x137f12e00>
        label = <none> 
        device = <AGXG13GDevice: 0x137dc5a00>
            name = Apple M1 
    retainedReferences = 1


235/250 [===========================>..] - ETA: 3s - loss: 0.3856 - accuracy: 0.8842 - f1_score: 0.8841

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x3d74d2e90>
    label = <none> 
    device = <AGXG13GDevice: 0x137dc5a00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x137f12e00>
        label = <none> 
        device = <AGXG13GDevice: 0x137dc5a00>
            name = Apple M1 
    retainedReferences = 1


250/250 [==============================] - 53s 213ms/step - loss: 0.3799 - accuracy: 0.8859 - f1_score: 0.8859
Epoch 3/100
 13/250 [>.............................] - ETA: 42s - loss: 0.2234 - accuracy: 0.9471 - f1_score: 0.9473

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x3f4cde460>
    label = <none> 
    device = <AGXG13GDevice: 0x137dc5a00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x137f12e00>
        label = <none> 
        device = <AGXG13GDevice: 0x137dc5a00>
            name = Apple M1 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x3f2c2dd40>
    label = <none> 
    device = <AGXG13GDevice: 0x137dc5a00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x137f12e00>
        label = <none> 
        device = <AGXG13GDevice: 0x137dc5a00>
            name = Apple M1 
    retainedReference

250/250 [==============================] - 48s 194ms/step - loss: 0.2284 - accuracy: 0.9321 - f1_score: 0.9322
Epoch 4/100
250/250 [==============================] - 46s 183ms/step - loss: 0.1839 - accuracy: 0.9446 - f1_score: 0.9447
Epoch 5/100
250/250 [==============================] - 45s 182ms/step - loss: 0.1534 - accuracy: 0.9499 - f1_score: 0.9499
Epoch 6/100
250/250 [==============================] - 41s 166ms/step - loss: 0.1343 - accuracy: 0.9569 - f1_score: 0.9569
Epoch 7/100
250/250 [==============================] - 41s 166ms/step - loss: 0.1209 - accuracy: 0.9645 - f1_score: 0.9645
Epoch 8/100
250/250 [==============================] - 40s 162ms/step - loss: 0.1128 - accuracy: 0.9653 - f1_score: 0.9653
Epoch 9/100
250/250 [==============================] - 41s 163ms/step - loss: 0.1001 - accuracy: 0.9704 - f1_score: 0.9704
Epoch 10/100
250/250 [==============================] - 37s 146ms/step - loss: 0.0961 - accuracy: 0.9705 - f1_score: 0.9705
Epoch 11/100
250/250 [=====